# Предсказание риска поражения сердца

## <u>Введение</u>

Целью проекта является создание модели машинного обучения, которая позволит осуществлять предсказание риска поражения сердца на основе данных о пациентах, а также подготовить библиотеку и интерфейс к ней для предсказания на тестовой выборке.

Для создания самой модели машинного обучения, необходимо решить следующие задачи:
1. Загрузить тренировочные данные и изучить информацию о них
2. Осуществить предварительную обработку данных
3. Выполнить исследовательский и корреляционный анализ данных
4. Подготовить обучающую выборку
5. Выбрать релевантные метрики для модели
6. Осуществить поиск наилучшей модели с подбором гиперпараметров на кросс-валидации.
7. Выполнить проверку модели

## <u>Импорт библиотек</u>

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
import re
import phik
import time
from typing import List

# графика
import matplotlib.pyplot as plt
import seaborn as sns


# модели
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# выборки
from sklearn.model_selection import train_test_split

# предобработка
from sklearn.preprocessing import (OneHotEncoder, 
                                   OrdinalEncoder, 
                                   StandardScaler, 
                                   MinMaxScaler, 
                                   RobustScaler)

from sklearn.impute import SimpleImputer

# пайплайны
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import NearMiss

# метрика
from sklearn.metrics import (accuracy_score, 
                            precision_score, 
                            recall_score, 
                            f1_score, 
                            roc_auc_score,
                            average_precision_score,
                            confusion_matrix,
                            precision_recall_curve)

# подбор гиперпараметров RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

# для оценки важности признаков
import shap

# предупреждения
import warnings

In [ ]:
import tempfile
tempfile.tempdir = 'C:/temp'

In [ ]:
# ratio тестовой выборки
TEST_SIZE = 0.25

In [ ]:
# создание константы RANDOM_STATE
RANDOM_STATE = 42 # nice ref to Douglas Adams :)

In [ ]:
# уберем предупреждения
warnings.filterwarnings(action="ignore")

## <u>Вспомогательный функции</u>

**Комментарий**: ниже представлены вспомогательные функции, которые разрабатывались и затем дорабатывались в рамках предыдущих проектов.

### Функция ```dataset_info```

In [ ]:
# функция вывода общей информации о датасете
def dataset_info(data):
    """
    Функция выводит информацию о датафрейме info, describe, первые и последние 5 записей
    
    Параметры:
    ----------
        data -- датафрейм
    
    Возвращает:
    -----------
        None
    """
    print("Общая информация о датасете:")
    data.info();
    print("Описательная статистика датасета:")
    display(data.describe())
    print("Первые 5 записей:")
    display(data.head())
    print("Последние 5 записей:")
    display(data.tail())

### Функция ```plot_isna```

In [ ]:
# функция для отображения количества пропусков в процентном соотношении от общего числа записей
def plot_isna(data):
    try:
        # вычисляем процент пропусков
        missing_data = data.isna().mean() * 100
        # фильтруем столбцы с пропусками и сортируем
        missing_data = missing_data[missing_data > 0].sort_values(ascending=True)
        
        if missing_data.empty:
            print('Пропуски в данных отсутствуют.')
            return
        
        # создаем горизонтальную столбчатую диаграмму
        fig, ax = plt.subplots(figsize=(13, 6))
        ax.barh(missing_data.index, missing_data, color='skyblue', edgecolor='black')
        
        # настраиваем подписи и заголовок
        ax.set_title('% пропусков в данных по столбцам\n(от общего количества записей в данных)', fontsize=14)
        ax.set_xlabel('Пропуски в данных, (%)', fontsize=12)
        ax.set_ylabel('Столбцы', fontsize=12)
        
        # настраиваем шрифт для меток на осях
        ax.tick_params(axis='both', labelsize=12)
        
        # добавляем сетку
        ax.grid(True, axis='x', linestyle='--', alpha=0.7)
        
        # строим график
        plt.show()
        print("Доля пропусков по столбцам:")
        display(missing_data)
        
    except Exception as e:
        print(f'Ошибка: {e}')

### Функция ```to_snake_case```

In [ ]:
def to_snake_case(name):
    """
    Преобразует CamelCase в snake_case:
    "DateCrawled" -> "date_crawled"
    
    Параметры:
    ----------
        name -- название столбца
        
    Возвращает:
    -----------
        name -- преобразованное в snake_case из CamelCase название столбца
    """
    
    # заменяем пробелы и спецсимволы на подчеркивания
    name = re.sub(r'[\s\-()]', '_', name)
    
    # вставляем _ только между camelCase (строчная -> заглавная)
    name = re.sub(r'([a-z])([A-Z])', r'\1_\2', name)
    
    # приводим к нижнему регистру и убираем множественные подчеркивания
    name = name.lower()
    name = re.sub(r'_+', '_', name)
    
    # убираем подчеркивания в начале/конце
    return name.strip('_')

### Функция ```dataset_duplicates_info```

In [ ]:
# функция для вывода общей информации о дубликатах в датафрейме
def dataset_duplicates_info(data):
    """
    Функция выводит информацию о количестве явных дубликатов в датафрейме 
    и кол-во уникальных значений в столбцах
    
    Параметры:
    ----------
        data -- датафрейм
    
    Возвращает:
    ----------- 
        None
    """
    print('Размерность данных:',data.shape)
    print(data.nunique())
    print('Количество явных дубликатов:',data.duplicated().sum())

### Функция ```get_duplicated_data```

In [ ]:
# функция для вывода явных дубликатов в датафрейме по столбцу
def get_duplicated_data(data, column: str = None):
    """
    Функция возвращает данные, в которых есть явные дубликаты в столбце column
    
    Аргументы:
        data -- датафрейм
        column -- столбец, в котором ищем явные дубликаты
    
    
    Возвращает: датафрейм, который содержит записи, где в column явные дубликаты
    """
    if column is None:
        return data[data.duplicated(keep=False)]
    else:
        return data[data[column].duplicated(keep=False)]

### Функция ```remove_duplicated_data```

In [ ]:
# функция для вывода явных дубликатов в датафрейме по столбцу
def remove_duplicated_data(data, column: str = None, inplace: bool = True):
    """
    Функция удаляет явные дубликаты в данных
    
    Аргументы:
        data -- датафрейм
        inplace -- флаг на удаление данных в непосредственно в исходной таблице
    
    
    Возвращает: 
        датафрейм без дубликатов если inplace=False
        None если inplace=True
    """
    print('Размерность данных до удаления дубликатов:', data.shape)
    initial_count = data.shape[0]
    
    if inplace:
        data.drop_duplicates(subset=column, inplace=True)
        removed_count = initial_count - data.shape[0]
        print(f'Удалено дубликатов: {removed_count}')
        print('Размерность данных после удаления дубликатов:', data.shape)
        return None
    else:
        result = data.drop_duplicates(subset=column)
        removed_count = initial_count - result.shape[0]
        print(f'Удалено дубликатов: {removed_count}')
        print('Размерность данных после удаления дубликатов:', result.shape)
        return result

### Функция ```plot_data_analysis```

In [ ]:
# вспомогательная функция для отрисовки параметров с целью их дальнейшего анализа
# функция была написана ранее в рамках предыдущих проектов, эта версия доработанная
def plot_data_analysis(
    data,
    title: str = 'Изучение параметров',
    title_box: str = 'Диаграмма размаха',
    title_hist: str = 'Распределение',
    x_label: str = 'X',
    y_label: str = 'Y',
    plot_box: bool = True,
    plot_bar: bool = False,
    x=None,
    y=None,
    bins: int = 200,
    bar_labels=None,
    discrete=False,
    figsize=None,
    color: str = 'skyblue',
    show_stats: bool = True,
    label_angle: int = 0,
    log: bool = False
):
    """
    Функция для отрисовки данных с возможностью построения:
        boxplot (диаграмма размаха)
        гистограммы или barplot (столбчатой диаграммы)
    
    Параметры:
    ----------
        data -- данные для анализа (pd.Series, pd.DataFrame или массив)
        title -- общий заголовок
        title_box -- заголовок для boxplot
        title_hist -- заголовок для гистограммы/barplot
        x_label -- подпись оси X
        y_label -- подпись оси Y
        plot_box -- строить ли boxplot (по умолчанию True)
        plot_bar -- использовать ли barplot вместо гистограммы (по умолчанию False)
        x -- данные для оси X (если нужен barplot с внешними данными)
        y -- данные для оси Y (если нужен barplot с внешними данными)
        bins -- количество бинов для гистограммы
        bar_labels -- подписи для barplot
        discrete -- флаг для дискретных данных
        figsize -- размер графика
        color -- основной цвет графиков
        show_stats -- показывать ли статистику (по умолчанию True)
    """
    # проверка и преобразование типов ходных данных
    if not isinstance(data, (pd.Series, pd.DataFrame, np.ndarray, list)):
        raise TypeError("Данные должны быть типа pd.Series, pd.DataFrame, np.ndarray или list")
    
    if isinstance(data, (pd.DataFrame, np.ndarray, list)):
        data = pd.Series(data)
    
    # настройка размера графика
    if figsize is None:
        figsize = (13, 8) if plot_box else (13, 6)
    
    # создание subplots
    if plot_box:
        fig, (ax_box, ax_main) = plt.subplots(2, 1, figsize=figsize, 
                                             gridspec_kw={'height_ratios': [1, 2]})
        # усы
        ax_box.boxplot(data, vert=False, patch_artist=True,
                      boxprops=dict(facecolor='lightblue'))
        ax_box.set_title(title_box, pad=10)
        ax_box.set_xlabel(x_label)
        ax_box.grid(axis='x', linestyle='--', alpha=0.7)
        ax_box.set_yticks([])
    else:
        fig, ax_main = plt.subplots(1, 1, figsize=figsize)
    
    # выбор типа основного графика
    if plot_bar:
        if x is None or y is None:
            if discrete:
                value_counts = data.value_counts().sort_index()
                x = value_counts.index
                y = value_counts.values
                bar_labels = x if bar_labels is None else bar_labels
            else:
                raise ValueError("Для barplot нужно указать x и y или использовать discrete=True")
        
        ax_main.bar(x=x, height=y, color=color, edgecolor='black', alpha=0.8)
        ax_main.set_title(title_hist, pad=10)
        
        # установка подписей для дискретных данных
        if bar_labels is not None:
            ax_main.set_xticks(x if discrete else np.arange(len(x)))
            ax_main.set_xticklabels(bar_labels, rotation=label_angle, ha='right')
    else:
        # гистограмма
        ax_main.hist(data, bins=bins, color=color, edgecolor='black', alpha=0.8, log=log)
        ax_main.set_title(title_hist, pad=10)
    
    # подписи к осям и сетка
    ax_main.set_xlabel(x_label)
    ax_main.set_ylabel(y_label)
    ax_main.grid(axis='both', linestyle='--', alpha=0.5)
    
    fig.suptitle(title, fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # вывод статистики
    if show_stats:
        display(data.describe())

### Функция ```plot_corr_heatmap```

In [ ]:
# функция построения тепловой карты коэффициентов корреляции Пирсона
def plot_corr_heatmap(data, 
                      title: str = 'Тепловая карта корреляции', 
                      columns: List[str] = None):
    """
    Функция для построения тепловой карты коэффициентов корреляции Пирсона
    
    Параметры:
    ----------
        data -- DataFrame
    """
    plt.figure(figsize=(14, 6))
    corr_matrix = data.phik_matrix(interval_cols=columns)
    sns.heatmap(corr_matrix, 
                annot=True,  # показывать значения в ячейках
                fmt=".2f",   # формат чисел (2 знака после запятой)
                cmap='coolwarm',  # цветовая схема
                vmin=-1, vmax=1,  # диапазон значений
                linewidths=0.5)   # ширина линий между ячейками

    # заголовок
    plt.title(title)

    # вывод графика
    plt.tight_layout()
    plt.show()
    return corr_matrix

### Функция ```plot_feature_vs_target```

In [ ]:
def plot_feature_vs_target(data, 
                           feature, 
                           target,
                           x_label='Признак',
                           y_label='Количество',
                           title='Распределение признака от целевого',
                           bins=30,
                           label_angle=0):
    """
    Наложенные распределения фичи по бинарному таргету
    
    Параметры:
    ----------
    data : DataFrame
        Датафрейм с данными
    feature : str
        Название фичи для анализа
    target : str
        Название бинарного целевого признака
    x_label: str
        Подпись оси X
    y_label: str
        Подпись оси Y
    title: str
        Заголовок графика
    bins: int
        Количество бинов для гистограммы
    label_angle: int
        Угол наклона подписей шкалы по оси X
    """
    
    plt.figure(figsize=(12, 6))
    
    # разделяем данные по целевому
    data_0 = data[data[target] == 0][feature]
    data_1 = data[data[target] == 1][feature]
    
    # непрерывных фич строим гистограммы
    if data[feature].dtype in ['int64', 'float64'] and data[feature].nunique() > 10:
        plt.hist(data_0, alpha=0.7, label='нет риска - (0)', bins=bins, color='skyblue', edgecolor='black')
        plt.hist(data_1, alpha=0.7, label='есть риск - (1)', bins=bins, color='salmon', edgecolor='black')
        plt.ylabel(y_label)
    
    # для категориальных/дискретных - bar plot
    else:
        counts_0 = data_0.value_counts().sort_index()
        counts_1 = data_1.value_counts().sort_index()
        
        x = np.arange(len(counts_0))
        width = 0.35
        
        bars1 = plt.bar(x - width/2, counts_0, width, label='нет риска - (0)', color='skyblue', alpha=0.8, edgecolor='black')
        bars2 = plt.bar(x + width/2, counts_1, width, label='есть риск - (1)', color='salmon', alpha=0.8, edgecolor='black')
        plt.bar_label(bars1, fmt='%.2f', padding=3, fontsize=8)
        plt.bar_label(bars2, fmt='%.2f', padding=3, fontsize=8)
        try:
            xtick_labels = [f'{float(val):.2f}' for val in counts_0.index]
        except (ValueError, TypeError):
            xtick_labels = [str(val) for val in counts_0.index]
        plt.xticks(x, xtick_labels, rotation=label_angle)
        plt.ylabel(y_label)
    
    plt.xlabel(x_label)
    plt.title(title)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # статистика
    print(f"Статистика по {feature}:")
    print(f"Уникальных значений: {data[feature].nunique()}")
    print(f"\nСредние значения:")
    
    # средние значения только для числовых фич
    if data[feature].dtype in ['int64', 'float64']:
        print(f"\nСредние значения:")
        print(f"Без риска: {data_0.mean():.6f}")
        print(f"С риском: {data_1.mean():.6f}")
    
    # разница средних для количественных
    if data[feature].nunique() > 10:
        mean_diff = data_1.mean() - data_0.mean()
        print(f"Разница средних (риск - нет риска): {mean_diff:.6f}")
        print(f"Относительная разница: {mean_diff/data_0.mean():.2%}")

    # доли риска для категориальных
    if data[feature].nunique() <= 10:
        risk_ratios = data.groupby(feature)[target].mean()
        print(f"\nДоли риска по категориям {feature}:")
        for category, ratio in risk_ratios.items():
            print(f"  {category}: {ratio:.1%}")

### Функция ```corr_analysis```

In [ ]:
def corr_analysis(corr_matrix, 
                  target_feature, 
                  features=None, 
                  min_abs_corr=0.1):
    """
    Анализирует корреляции целевого признака с другими признаками и сортирует их по убыванию.
    
    Параметры:
    ----------
    corr_matrix : DataFrame
        Матрица корреляции (phik-матрица или обычная корреляционная матрица)
    target_feature : str
        Название целевого признака для анализа
    features : list, optional
        Список признаков для анализа (если None, берутся все из матрицы)
    min_abs_corr : float, optional
        Минимальная абсолютная корреляция для включения в результат (по умолчанию 0.1)
        
    Возвращает:
    -----------
    DataFrame
        Таблица с признаками, отсортированными по убыванию абсолютной корреляции с целевым признаком,
        с указанием силы связи по шкале Чеддока
    """
    # если не указан список признаков, берем все из матрицы (исключая целевой)
    if features is None:
        features = [col for col in corr_matrix.columns if col != target_feature]
    
    # собираем корреляции с целевым признаком
    correlations = []
    
    for feature in features:
        if feature == target_feature:
            continue
        
        corr = corr_matrix.loc[target_feature, feature]
        abs_corr = abs(corr)
        
        # реализуем шкалу Чеддока (согласно 11 спринту - 3 тема - 9 урок)
        if abs_corr >= 0.9:
            strength = 'Весьма высокая'
        elif abs_corr >= 0.7:
            strength = 'Высокая'
        elif abs_corr >= 0.5:
            strength = 'Заметная'
        elif abs_corr >= 0.3:
            strength = 'Умеренная'
        elif abs_corr >= 0.1:
            strength = 'Слабая'
        elif abs_corr > min_abs_corr and abs_corr < 0.1:
            strength = 'Очень слабая'
        elif abs_corr < min_abs_corr:
            strength = 'Отсутствует'
        
        # определяем направление связи
        direction = 'положительная' if corr > 0 else 'отрицательная'
        
        correlations.append({
            'Признак': feature,
            'Корреляция': corr,
            'Абс. корреляция': abs_corr,
            'Сила связи': strength,
            'Направление': direction
        })
    
    # создаем DataFrame и сортируем по убыванию абсолютной корреляции
    if not correlations:
        return pd.DataFrame()  # возвращаем пустой DataFrame если нет корреляций
    
    result_df = pd.DataFrame(correlations).sort_values(
        by='Абс. корреляция', ascending=False
    ).reset_index(drop=True)
    
    # вывод корреляции для удобства чтения
    result_df['Корреляция'] = result_df['Корреляция'].apply(lambda x: f"{x:.3f}")
    
    return result_df[['Признак', 'Корреляция', 'Сила связи', 'Направление']]

### Функция ```evaluate_model```

In [ ]:
def evaluate_model(model, 
                   params, 
                   model_name, 
                   X_train, 
                   y_train,
                   preprocessor_ohe,
                   preprocessor_ordinal,
                   random_state=42,
                   refit_metric='roc_auc',
                   cv=3,
                   n_iter=5):
    """
    Функция для оценки моделей бинарной классификации
    
    Параметры:
    ----------
    model: модель
    params: гиперпараметры для перебора
    model_name: название модели
    X_train: входные признаки обучающей выборки
    y_train: целевой признак обучающей выборки
    preprocessor: пайплайн для предобработки
    random_state: зерно для рандома
    refit_metric: метрика для выбора лучшей модели 
                 ('roc_auc', 'recall', 'precision', 'f1', 'average_precision')
    cv: количество cv фолдов
    n_iter: количество итераций
    
    Возвращает:
    -----------
    Словарь c параметрами:  
        - 'model_name': название модели
        - 'best_model': лучшая модель после подбора гиперпараметров
        - 'best_params': лучшие гиперпараметры
        - 'accuracy_cv': Accuracy на кросс-валидации
        - 'precision_cv': Precision на кросс-валидации
        - 'recall_cv': Recall на кросс-валидации
        - 'roc_auc_cv': ROC-AUC на кросс-валидации
        - 'average_precision_cv': Average Precision на кросс-валидации
        - 'f1_cv': F1-score на кросс-валидации
        - 'params_time': время подбора гиперпараметров
        - 'train_time': среднее время обучения лучшей модели (из cv_results_df)
        - 'predict_time': среднее время предсказания лучшей модели (из cv_results_df)
        - 'cv_results_df': таблица с полной информацией из cv_results_
    """
    
    linear_models = ['LogisticRegression', 'KNeighborsClassifier']
    
    is_linear = any(linear_model in model_name for linear_model in linear_models)
    
    if 'Dummy' in model_name:
        preprocessor = preprocessor_ordinal
    elif any(linear_model in model_name for linear_model in linear_models):
        preprocessor = preprocessor_ohe
        print(f"Для линейной модели {model_name} используется OneHotEncoder")
    else:
        preprocessor = preprocessor_ordinal
        print(f"Для нелинейной модели {model_name} используется OrdinalEncoder")
    
    pipeline = Pipeline(
        steps = [
            ('preprocessor', preprocessor),
            ('model', model)
        ]
    )
    
    # метрики для классификации
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall', 
        'roc_auc': 'roc_auc',
        'f1': 'f1',
        'average_precision': 'average_precision'
    }
    
    # Проверяем что refit_metric есть в scoring
    if refit_metric not in scoring:
        raise ValueError(f"refit_metric должен быть одним из: {list(scoring.keys())}")
    
    # замер времени подбора параметров
    search = RandomizedSearchCV(pipeline, 
                                params, 
                                n_iter=n_iter, 
                                cv=cv,
                                scoring=scoring,
                                refit=refit_metric,
                                return_train_score=True,
                                random_state=random_state,
                                n_jobs=-1)
    
    try:
        # замер времени подбора параметров
        start_time = time.time()
        search.fit(X_train, y_train)
        params_time = time.time() - start_time
    except Exception as e:
        print(f"Ошибка при подборе гиперпараметров для {model_name}: {e}")
        return None

    # берем cv_results_
    cv_results_df = pd.DataFrame(search.cv_results_)
    
    best_index = search.best_index_
    best_train_time = cv_results_df.loc[best_index, 'mean_fit_time']
    best_predict_time = cv_results_df.loc[best_index, 'mean_score_time']
    
    # берем лучшие метрики
    best_accuracy_cv = cv_results_df.loc[best_index, 'mean_test_accuracy']
    best_precision_cv = cv_results_df.loc[best_index, 'mean_test_precision']
    best_recall_cv = cv_results_df.loc[best_index, 'mean_test_recall']
    best_roc_auc_cv = cv_results_df.loc[best_index, 'mean_test_roc_auc']
    best_f1_cv = cv_results_df.loc[best_index, 'mean_test_f1']
    best_average_precision_cv = cv_results_df.loc[best_index, 'mean_test_average_precision']

    print(f"Модель: {model_name}")
    print(f"Лучшие параметры: {search.best_params_}")
    print(f"Время подбора параметров: {params_time:.3f} с.")
    print(f"Время обучения лучшей модели: {best_train_time:.3f} с.")
    print(f"Время предсказания лучшей модели: {best_predict_time:.3f} с.")
    print(f"Accuracy на кросс-валидации: {best_accuracy_cv:.4f}")
    print(f"Precision на кросс-валидации: {best_precision_cv:.4f}")
    print(f"Recall на кросс-валидации: {best_recall_cv:.4f}")
    print(f"ROC-AUC на кросс-валидации: {best_roc_auc_cv:.4f}")
    print(f"F1-score на кросс-валидации: {best_f1_cv:.4f}")
    print(f"Average Precision на кросс-валидации: {best_average_precision_cv:.4f}\n")

    return {
        'model_name': model_name,
        'best_model': search.best_estimator_,
        'best_params': search.best_params_,
        'accuracy_cv': best_accuracy_cv,
        'precision_cv': best_precision_cv,
        'recall_cv': best_recall_cv,
        'roc_auc_cv': best_roc_auc_cv,
        'average_precision_cv': best_average_precision_cv,
        'f1_cv': best_f1_cv,
        'params_time': params_time,
        'train_time': best_train_time,
        'predict_time': best_predict_time,
        'cv_results_df': cv_results_df
    }

### Функция ```plot_confusion_matrx```

In [ ]:
def plot_confusion_matrix(conf_matrix, 
                          title='Матрица ошибок',
                          labels=['нет', 'да']
                         ):
    """
    Визуализирует матрицу ошибок (confusion matrix)
    
    Параметры:
    ----------
    conf_matrix : array-like
        Матрица ошибок в формате [[TN, FP], [FN, TP]]
    title : str, optional
        Заголовок графика (по умолчанию 'Матрица ошибок')
    labels : list, optional
        Подписи классов [negative, positive] (по умолчанию ['нет', 'да'])
    """
    plt.figure(figsize=(13, 6))
    ax = sns.heatmap(conf_matrix, 
                     annot=True, 
                     fmt='d', 
                     cmap='icefire',
                     xticklabels=labels,
                     yticklabels=labels)
    
    ax.set_title(title, pad=20, fontsize=12)
    ax.set_xlabel('Предсказанное значение', fontsize=10)
    ax.set_ylabel('Реальное значение', fontsize=10)
    
    # добавляем аннотации для лучшей читаемости
    for i in range(len(conf_matrix)):
        for j in range(len(conf_matrix[0])):
            text = ax.texts[i * len(conf_matrix[0]) + j]
            text.set_fontsize(12)
            text.set_fontweight('bold')
    
    plt.tight_layout()
    plt.show()

    # дополнительная текстовая интерпретация
    tn, fp, fn, tp = conf_matrix.ravel()
    print(f"Интерпретация:")
    print(f"  Правильно предсказано 'нет' (True Negative): {tn}")
    print(f"  Ложно предсказано 'да' (False Positive): {fp}")
    print(f"  Ложно предсказано 'нет' (False Negative): {fn}")
    print(f"  Правильно предсказано 'да' (True Positive): {tp}")

### Класс модели с подбором порога

In [ ]:
class Classifier:
    def __init__(self, base_model, threshold=0.352):
        self.base_model = base_model
        self.threshold = threshold
        
    def predict(self, X):
        if hasattr(self.base_model, 'predict_proba'):
            proba = self.base_model.predict_proba(X)[:, 1]
        else:
            decision = self.base_model.decision_function(X)
            proba = (decision - decision.min()) / (decision.max() - decision.min())
        return (proba > self.threshold).astype(int)
    
    def predict_proba(self, X):
        return self.base_model.predict_proba(X)
    
    def fit(self, X, y):
        return self.base_model.fit(X, y)

## <u>Загрузка данных и изучение общей информации</u>

### Загрузка данных

**Комментарий**: выполним загрузку данных ```heart_train.csv```.

In [ ]:
# выполняем чтение данных
# также снимем ограничение на выводимое кол-во столбцов
pd.options.display.max_columns = None
pd.options.display.max_rows = None
try:
    heart = pd.read_csv('data\\heart_train.csv', index_col=[0])
except:
    print("Ошибка чтения данных!")

In [ ]:
# выведем общую информацию о данных
dataset_info(heart)

### Промежуточный вывод

Данные содержат 8684 записи с информацией (**27** признаков) о пациентах.

Важные моменты:
1. В данных присутствуют пропуски;
2. Можно предположить, что большая часть количественных данных видимо уже отмасштабирована (например, возраст);
3. Необходимо привести именование столбцов к `snake_case` формату;
4. Необходимо проверить типы данных в столбцах;

Также важно рассмотреть несколько подробнее признаки, представленные в данных:

|#|Признак|Описание|
|--|:-|-:|
|1|`id`|уникальный идентификатор пациента|
|2|`Age`|возраст пациента|
|3|`Cholesterol`|уровень холестерина в крови пациента|
|4|`Heart Rate`|пульс пациента|
|5|`Diabetes`|наличие у пациента диагностированного диабета (`0` -- нет, `1` -- да)|
|6|`Family History`|были ли проблемы с сердцем у родственников пациента (`0` -- нет, `1` -- да)|
|7|`Smoking`|является ли пациент курильщиком (`0` -- нет, `1` -- да)|
|8|`Obesity`|страдает ли пациент от ожирения (`0` -- нет, `1` -- да)|
|9|`Alcohol Consumption`|употребление алкоголя пациентом (`0` -- нет, `1` -- да)|
|10|`Exercise Hours Per Week`|количество часов физических упражнений в неделю|
|11|`Diet`|какая у пациента диета/питание|
|12|`Previous Heart Problems`$^1$|были ли ранее у пациента проблемы с сердцем (`0` -- нет, `1` -- да)|
|13|`Medication Use`|принимает ли пациент лекарственные препараты (`0` -- нет, `1` -- да)|
|14|`Stress Level`|уровень стресса пациента (десятибалльная шкала)|
|15|`Sedentary Hours Per Day`|сколько часов в день проводит пациент в сидячем положении|
|16|`Income`|уровень дохода пациента|
|17|`BMI`|ИМТ -- индекс массы тела пациента|
|18|`Triglycerides`$^2$|уровень триглициридов в крови пациента|
|19|`Physical Activity Days Per Week`|количество дней в неделю с физической активностью|
|20|`Sleep Hours Per Day`|количество часов сна в сутки|
|21|`Heart Attack Risk (Binary)`|риск сердечного приступа у пациента **целевой признак** (`0` -- нет, `1` -- да)|
|22|`Blood Sugar`$^3$|уровень сахара в крови пациента|
|23|`CK-MB`$^4$|содержание CK-MB в крови пациента|
|24|`Troponin`$^5$|содержание тропонина в крови пациента|
|25|`Gender`| пол пациента (`Male` -- мужской, `Female` -- женский)|
|26|`Systolic Blood Pressure`$^6$|cистолическое артериальное давление пациента|
|27|`Diastolic Blood Pressure`$^7$|диастолическое артериальное давление пациента|

**Пояснения по некоторым признакам по сноскам**:

$^1$ -- признак, связанный с ранее выявленными проблемами с сердцем является потенциальной утечкой, особенно для случая первичной оценки риска. Нужно учесть это при отборе признаков.

$^2$ -- `Триглицериды` -- это тип жиров в крови, являющийся основным источником энергии для организма и запасающимся в жировой ткани. После еды их уровень повышается, поскольку организм накапливает избыток калорий. Высокий уровень триглицеридов увеличивает риск атеросклероза, инфаркта и инсульта из-за образования жировых отложений на стенках сосудов. 

$^3$ -- Высокий уровень сахара в крови напрямую связан с проблемами сердца, так как повреждает кровеносные сосуды, способствует образованию жировых отложений и атеросклерозу, что повышает риск инфарктов, инсультов, сердечной недостаточности и других серьезных сердечно-сосудистых заболеваний.

$^4$ -- `CK-MB` (или КФК-МВ) -- это сокращение для обозначения креатинкиназы-МВ, специфического фермента, который находится преимущественно в клетках сердечной мышцы. Анализ на CK-MB используется в медицине как кардиальный биомаркер для диагностики повреждений миокарда, таких как инфаркт, поскольку ***при разрушении сердечной мышцы этот фермент высвобождается в кровь***. **Невоспроизводимый признак** -- нужно учесть при отборе признаков для обучения модели.

$^5$ -- `Тропонин` -- это белок, содержащийся в сердечной мышце и скелетной мускулатуре, необходимый для мышечного сокращения, который является высокоспецифичным маркером повреждения сердца. При инфаркте миокарда клетки сердечной мышцы разрушаются, и тропонин попадает в кровоток, поэтому ***его повышенный уровень в крови указывает на сердечную патологию, чаще всего инфаркт миокарда***. **Невоспроизводимый признак** -- нужно учесть при отборе признаков для обучения модели.

$^6$ -- `Систолическое артериальное давление` -- это показатель максимального давления в артериях, возникающего в момент сокращения сердечной мышцы (систолы). Оно является первой, верхней цифрой в записи артериального давления (например, 120/80 мм рт. ст.) и отражает силу и скорость сокращений сердца. Характеризует состояние миокарда (сердечной мышцы) и силу, с которой сердце выталкивает кровь в артерии. 

$^7$ -- `Диастолическое артериальное давление` -- это "нижняя" цифра артериального давления, которая показывает давление в сосудах в момент расслабления сердечной мышцы между сокращениями. Нормальными считаются показатели диастолического давления в пределах 60–80 мм рт. ст. для взрослых, хотя эти значения могут варьироваться в зависимости от возраста. Отклонения диастолического давления могут указывать на проблемы с сосудами или сердцем.

## <u>Предобработка данных</u>

**Комментарий**: выполним предварительную обработку данных.

### Первичный отбор признаков

**Комментарий**: проведем первичный отбор признаков на основе анализа проведенного в разделе `Загрузка данных и изучение общей информации`. Необходимо убрать признаки `CK-MB` и `Troponin`, т.к. они являются не воспроизводимыми, а также уберем признак `id`, т.к. он неинформативен и признак `Previous Heart Problems`, т.к. это потенциальная утечка.

In [ ]:
# удалим столбцы с указанными признаками
print("Размерность данных до удаления признаков:", heart.shape)
heart.drop(['id', 'CK-MB', 'Troponin', 'Previous Heart Problems'], inplace=True, axis=1)
print("Размерность данных после удаления признаков:", heart.shape)

In [ ]:
# выведем обновленную информацию о данных
dataset_info(heart)

**Промежуточный вывод**: признаки `CK-MB` и `Troponin` убраны, т.к. они являются не воспроизводимыми, признак `id` убран, т.к. он неинформативен и признак `Previous Heart Problems` убран, т.к. это потенциальная утечка.

### Именование столбцов

**Комментарий**: проверим и исправим именование столбцов, приведем их к формату `snake_case`.

In [ ]:
print("Названия столбцов в данных до преобразования:", 
      heart.columns.to_list())

for col in heart.columns:
    heart.rename(columns=to_snake_case, inplace=True)
    
print("Названия столбцов в данных после преобразования:", 
      heart.columns.to_list())

**Промежуточный вывод**: выполнено переименование столбцов с приведением к `snake_case` формату.

### Пропуски в данных

**Комментарий**: проверим данные на пропуски.

In [ ]:
# посмотрим количество пропусков в данных
plot_isna(heart)

**Комментарий**: наблюдается $\sim2.8%$ пропусков в данных. Проблема в том, что данные очень чувствительны и индивидуальны, массово заполнить мы их не сможем, т.к. это может исказить информацию. Проверим, все ли пропуски соответствуют одним и тем же строкам. В случае положительного результата -- удалим их.

In [ ]:
# сформируем список столбцов, где найдены пропуски
na_cols = ['diabetes', 'family_history', 
           'smoking', 'obesity', 
           'alcohol_consumption',
           'medication_use', 'stress_level',
           'physical_activity_days_per_week']

# проверим, совпадают ли индексы пропусков
missing_mask = heart[na_cols].isna().any(axis=1)
print("Пропущено строк:", missing_mask.sum())

**Комментарий**: пропуски в одних и тех же строках, удалим.

In [ ]:
# если это одни и те же строки -- удаляем
if missing_mask.sum() <= missing_mask.sum():
    print("Размерность данных до удаления:", heart.shape)
    heart = heart.dropna(subset=na_cols)
    print("Размерность данных после удаления:", heart.shape)

In [ ]:
# проверим наличие пропусков еще раз
plot_isna(heart)

**Промежуточный вывод**: было обнаружено и удалено **243** строки в данных с пропусками ($\sim2.8$%).

### Типы данных

**Комментарий**: проверим и исправим (при необходимости) типы данных в столбцах.

In [ ]:
# выведем информацию о данных
dataset_info(heart)
# выведем информацию о кол-ве уникальных значений в столбцах
dataset_duplicates_info(heart)

**Комментарий**: теоретически мы можем установить тип данных для столбцов `diabetes`, `family_history`, `smoking`, `obesity`, `alcohol_consumption`, `previous_heart_problems`, `medication_use`, `stress_level`, `physical_activity_days_per_week`, `heart_attack_risk_binary` к типу `int`, т.к. это целочисленные значения.

In [ ]:
# сформируем список столбцов
int_cols = ['diabetes', 'family_history', 'smoking', 'obesity', 'alcohol_consumption', 
           'medication_use', 'stress_level', 
           'physical_activity_days_per_week', 'heart_attack_risk_binary']

heart[int_cols] = heart[int_cols].astype(int)

In [ ]:
# выведем информацию о данных
dataset_info(heart)

**Промежуточный вывод**: данные в столбцах `diabetes`, `family_history`, `smoking`, `obesity`, `alcohol_consumption`, `previous_heart_problems`, `medication_use`, `stress_level`, `physical_activity_days_per_week`, `heart_attack_risk_binary` приведены к типу `int`, т.к. это целочисленные значения.

### Дубликаты в данных

**Комментарий**: проверим данные на наличие явных дубликатов.

In [ ]:
# информация о дубликатах
dataset_duplicates_info(heart)

**Комментарий**: обнаружено **12** явных дубликатов.

In [ ]:
# выведем дубликаты, т.к. их немного
get_duplicated_data(heart).sort_values('age')

**Комментарий**: удалим обнаруженные явные дубликаты.

In [ ]:
# вызовем метод удаления явных дубликатов
remove_duplicated_data(heart)

In [ ]:
# выведем снова информацию о дубликатах
dataset_duplicates_info(heart)

**Промежуточный вывод**: удалено **12** явных дубликатов.

### Промежуточный вывод

Выполнена предварительная обработка данных:

1. Выполнен первичный отбор признаков: признаки `CK-MB` и `Troponin` убраны, т.к. они являются не воспроизводимыми, признак `id` убран, т.к. он неинформативен и признак `Previous Heart Problems` убран, т.к. это потенциальная утечка;
2. Выполнено переименование столбцов с приведением к `snake_case` формату.
3. В данных обнаружено и удалено **243** строки с пропусками ($\sim2.8$% от общего кол-ва записей);
4. Данные в столбцах `diabetes`, `family_history`, `smoking`, `obesity`, `alcohol_consumption`, `previous_heart_problems`, `medication_use`, `stress_level`, `physical_activity_days_per_week`, `heart_attack_risk_binary` приведены к типу `int`, т.к. это целочисленные значения;
5. В данных обнаружено и удалено **12** явных дубликатов.

## <u>Исследовательский анализ данных</u>

**Комментарий**: теперь проведем исследовательский анализ данных, в том числе на предмет аномальных и некорректных значений. Также пронализируем их связь и распределение по целевому признаку.

В данных присутствуют следующие типы признаков:

***Количественные***:
- `age` -- возраст пациента
- `cholesterol` -- уровень холестерина в крови пациента 
- `heart_rate` -- пульс пациента
- `exercise_hours_per_week` -- количество часов физических упражнений в неделю
- `sedentary_hours_per_day` -- сколько часов в день проводит пациент в сидячем положении
- `income` -- уровень дохода пациента
- `bmi` -- ИМТ -- индекс массы тела пациента
- `triglycerides` -- уровень триглициридов в крови пациента
- `physical_activity_days_per_week` -- количество дней в неделю с физической активностью
- `sleep_hours_per_day` -- количество часов сна в сутки
- `blood_sugar` -- уровень сахара в крови пациента
- `systolic_blood_pressure` -- cистолическое артериальное давление пациента
- `diastolic_blood_pressure` -- диастолическое артериальное давление пациента

***Категориальные***:
- `diabetes` -- наличие у пациента диагностированного диабета
- `family_history` -- были ли проблемы с сердцем у родственников пациента
- `smoking` -- является ли пациент курильщиком
- `obesity` -- страдает ли пациент от ожирения
- `alcohol_consumption` -- употребление алкоголя пациентом
- `diet` -- какая у пациента диета/питание
- `medication_use` -- принимает ли пациент лекарственные препараты
- `stress_level` -- уровень стресса пациента
- `gender` -- пол пациента

**Целевой признак**:
- `heart_attack_risk_binary` -- риск сердечного приступа у пациента **целевой признак**

### Количественные признаки

**Комментарий**: рассмотрим сначала количественные признаки:

- `age` -- возраст пациента
- `cholesterol` -- уровень холестерина в крови пациента 
- `heart_rate` -- пульс пациента
- `exercise_hours_per_week` -- количество часов физических упражнений в неделю
- `sedentary_hours_per_day` -- сколько часов в день проводит пациент в сидячем положении
- `income` -- уровень дохода пациента
- `bmi` -- ИМТ -- индекс массы тела пациента
- `triglycerides` -- уровень триглициридов в крови пациента
- `physical_activity_days_per_week` -- количество дней в неделю с физической активностью
- `sleep_hours_per_day` -- количество часов сна в сутки
- `blood_sugar` -- уровень сахара в крови пациента
- `systolic_blood_pressure` -- cистолическое артериальное давление пациента
- `diastolic_blood_pressure` -- диастолическое артериальное давление пациента

#### Признак `age`

**Комментарий**: исследуем признак `age` -- возраст пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['age'],
                   title='Возраст пациента',
                   x_label='Нормализованный возраст пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'age', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от возраста',
                       x_label='Нормализованный возраст пациента',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `cholesterol`

**Комментарий**: исследуем признак `cholesterol` -- уровень холестерина в крови пациента 

In [ ]:
# выведем график распределения
plot_data_analysis(heart['cholesterol'],
                   title='Уровень холестерина в крови пациента',
                   x_label='Нормализованный показатель уровня холестерина в крови пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'cholesterol', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от уровня холестерина в крови',
                       x_label='Нормализованный уровень холестерина в крови пациента',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `heart_rate`

**Комментарий**: исследуем признак `heart_rate` -- пульс пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['heart_rate'],
                   title='Пульс пациента',
                   x_label='Нормализованное значение пульса пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'heart_rate', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от возраста',
                       x_label='Нормализованный возраст пациента',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `exercise_hours_per_week`

**Комментарий**: исследуем признак `exercise_hours_per_week` -- количество часов физических упражнений в неделю.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['exercise_hours_per_week'],
                   title='Количество часов физических упражнений в неделю',
                   x_label='Нормализованное значение количества часов физических упражнений в неделю',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'exercise_hours_per_week', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от часов физических упражнений в неделю',
                       x_label='Нормализованное значение количества часов физических упражнений в неделю',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `sedentary_hours_per_day`

**Комментарий**: исследуем признак `sedentary_hours_per_day` -- сколько часов в день проводит пациент в сидячем положении.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['sedentary_hours_per_day'],
                   title='Сколько часов в день проводит пациент в сидячем положении?',
                   x_label='Нормализованное значение количества часов в сидячем положении',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'sedentary_hours_per_day', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от количества часов в сидячем положении',
                       x_label='Нормализованное значение количества часов в сидячем положении',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `income`

**Комментарий**: исследуем признак `income` -- уровень дохода пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['income'],
                   title='Уровень дохода пациента',
                   x_label='Нормализованное значение уровня дохода пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'income', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от уровня дохода',
                       x_label='Нормализованный значение уровня дохода пациента',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `bmi`

**Комментарий**: исследуем признак `bmi` -- ИМТ -- индекс массы тела пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['bmi'],
                   title='Индекс массы тела пациента',
                   x_label='Нормализованное значение ИМТ пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'bmi', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от ИМТ пациента',
                       x_label='Нормализованное значение ИМТ пациента',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `triglycerides`

**Комментарий**: исследуем признак `triglycerides` -- уровень триглициридов в крови пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['triglycerides'],
                   title='Уровень триглициридов в крови пациента',
                   x_label='Нормализованное значение триглициридов в крови пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'triglycerides', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от уровня триглициридов в крови пациента',
                       x_label='Нормализованное значение триглициридов в крови пациента',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `physical_activity_days_per_week`

**Комментарий**: исследуем признак `physical_activity_days_per_week` -- количество дней в неделю с физической активностью.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['physical_activity_days_per_week'],
                   title='Количество дней в неделю с физической активностью',
                   x_label='Количество дней в неделю с физической активностью',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=8
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'physical_activity_days_per_week', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости ' \
                       'от количества дней в неделю с физической активностью',
                       x_label='Количество дней в неделю с физической активностью',
                       y_label='Количество пациентов',
                       bins=8
                       )

**Промежуточный вывод**:

#### Признак `sleep_hours_per_day`

**Комментарий**: исследуем признак `sleep_hours_per_day` -- количество часов сна в сутки.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['sleep_hours_per_day'],
                   title='Количество часов сна в сутки',
                   x_label='Нормализованное значение количества часов сна в сутки',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=7
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'sleep_hours_per_day', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости ' \
                       'от количества часов сна в сутки',
                       x_label='Нормализованное значение количества часов сна в сутки',
                       y_label='Количество пациентов',
                       bins=7
                       )

**Промежуточный вывод**:

#### Признак `blood_sugar`

**Комментарий**: исследуем признак `blood_sugar` -- уровень сахара в крови пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['blood_sugar'],
                   title='Уровень сахара в крови пациента',
                   x_label='Нормализованное значение уровня сахара в крови пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=100
                   )

**Комментарий**: рассмотрим распределение в логарифмической шкале по количеству пациентов.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['blood_sugar'],
                   title='Уровень сахара в крови пациента',
                   x_label='Нормализованное значение уровня сахара в крови пациента',
                   y_label='Количество пациентов (log-шкала)',
                   discrete=False,
                   plot_box=False,
                   bins=100,
                   log = True
                   )

In [ ]:
# находим значение, которое встречатся чаще всего
value_counts = heart['blood_sugar'].value_counts()
most_common_val = value_counts.index[0]
most_common_count = value_counts.iloc[0]

print(f"Самое популярное значение: {most_common_val} ({most_common_count} пациентов)")

# ищем и берем ближайших соседей слева и справ
all_values_sorted = sorted(heart['blood_sugar'].unique())
target_idx = all_values_sorted.index(most_common_val)
left_neighbor = all_values_sorted[target_idx - 1] if target_idx > 0 else None
right_neighbor = all_values_sorted[target_idx + 1] if target_idx < len(all_values_sorted) - 1 else None

print(f"Соседи: слева={left_neighbor}, справа={right_neighbor}")

# считаем среднее количество пациентов у соседей
neighbor_counts = []
if left_neighbor is not None:
    left_count = (heart['blood_sugar'] == left_neighbor).sum()
    neighbor_counts.append(left_count)
    print(f"Левый сосед {left_neighbor}: {left_count} пациентов")
    
if right_neighbor is not None:
    right_count = (heart['blood_sugar'] == right_neighbor).sum()
    neighbor_counts.append(right_count)
    print(f"Правый сосед {right_neighbor}: {right_count} пациентов")

# получаем значение, сколько нам оставить пациентов от самого частого
n_to_keep = int(np.mean(neighbor_counts)) if neighbor_counts else min(50, most_common_count)
print(f"Оставляем {n_to_keep} случайных пациентов с blood_sugar = {most_common_val}")

# берем радномных пациков заданного количества
popular_indices = heart[heart['blood_sugar'] == most_common_val].index
np.random.seed(RANDOM_STATE)
keep_indices = np.random.choice(popular_indices, size=n_to_keep, replace=False)

# собираем df
heart = pd.concat([
    heart[heart['blood_sugar'] != most_common_val],
    heart.loc[keep_indices]
])

print(f"Было: {len(heart)} пациентов")
print(f"Стало: {len(heart)} пациентов")
print(f"Убрали {most_common_count - n_to_keep} пациентов с популярным значением")

In [ ]:
# выведем график распределения
plot_data_analysis(heart['blood_sugar'],
                   title='Уровень сахара в крови пациента',
                   x_label='Нормализованное значение уровня сахара в крови пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=100,
                   log = False
                   )

In [ ]:
# выведем значения распределения
result = heart.groupby(pd.cut(heart['blood_sugar'], bins=10))['heart_attack_risk_binary'].agg(['mean', 'count'])
print(result)

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Данные неестественно сконцентрированы вокруг значения $\sim$**0.227**. Кажется, что в реальности распределение сахара в крови должно быть более плавным. Однако, максимум на **0.227** может судить о том, что большинство пациентов имеет нормальное значение сахара в крови. Действительно, это подтверждается выводом данных выше: $\sim80$% пациентов сосредоточены в узком диапазоне [0.2, 0.3], при этом хвосты содержат слишком мало данных для надежных выводов по ним. Поэтому лучше использовать признак как есть и выбросы убирать категорически не стоит.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'blood_sugar', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости ' \
                       'от значения систолического артериального давления',
                       x_label='Нормализованное значение систолического артериального давления пациента',
                       y_label='Количество пациентов',
                       bins=30
                       )

**Промежуточный вывод**:

#### Признак `systolic_blood_pressure`

**Комментарий**: исследуем признак `systolic_blood_pressure` -- cистолическое артериальное давление пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['systolic_blood_pressure'],
                   title='Систолическое артериальное давление пациента',
                   x_label='Нормализованное значение систолического артериального давления пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'systolic_blood_pressure', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости ' \
                       'от значения систолического артериального давления',
                       x_label='Нормализованное значение систолического артериального давления пациента',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

#### Признак `diastolic_blood_pressure`

**Комментарий**: исследуем признак `diastolic_blood_pressure` -- диастолическое артериальное давление пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['systolic_blood_pressure'],
                   title='Диастолическое артериальное давление пациента',
                   x_label='Нормализованное значение диастолического артериального давления пациента',
                   y_label='Количество пациентов',
                   discrete=False,
                   plot_box=True,
                   bins=20
                   )

**Комментарий**: данные в этом столбце были уже предварительно отмасштабированы. Каких-то выбросов или аномалий не наблюдается. Посмотрим теперь распределение по целевому признаку.

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'systolic_blood_pressure', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости ' \
                       'от значения диастолического артериального давления',
                       x_label='Нормализованное значение диастолического артериального давления пациента',
                       y_label='Количество пациентов',
                       bins=20
                       )

**Промежуточный вывод**:

### Категориальные признаки

**Комментарий**: теперь рассмотрим категориальные признаки:
- `diabetes` -- наличие у пациента диагностированного диабета
- `family_history` -- были ли проблемы с сердцем у родственников пациента
- `smoking` -- является ли пациент курильщиком
- `obesity` -- страдает ли пациент от ожирения
- `alcohol_consumption` -- употребление алкоголя пациентом
- `diet` -- какая у пациента диета/питание
- `previous_heart_problems` -- были ли ранее у пациента проблемы с сердцем
- `medication_use` -- принимает ли пациент лекарственные препараты
- `stress_level` -- уровень стресса пациента
- `gender` -- пол пациента

#### Признак `diabetes`

**Комментарий**: рассмотрим признак `diabetes` -- наличие у пациента диагностированного диабета.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['diabetes'],
                   title = 'Пациенты с диагностированным диабетом',
                   x_label = 'Наличие диабета (0 -- нет, 1 -- да)',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'diabetes', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от наличия диабета',
                       x_label='Наличие диабета (0 -- нет, 1 -- да)',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Признак `family_history`

**Комментарий**: рассмотрим признак `family_history` -- были ли проблемы с сердцем у родственников пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['family_history'],
                   title = 'Количество пациентов с наличием проблем с сердцем у родственников',
                   x_label = 'Наличие проблем с сердцем у родственников (0 -- нет, 1 -- да)',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'family_history', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от наличия проблем с сердцем у родственников',
                       x_label='Наличие проблем с сердцем у родственников (0 -- нет, 1 -- да)',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Признак `smoking`

**Комментарий**: рассмотрим признак `smoking` -- является ли пациент курильщиком.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['smoking'],
                   title = 'Пациенты курильщики',
                   x_label = 'Курит ли пациент (0 -- нет, 1 -- да)',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'smoking', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от курения',
                       x_label='Курит ли пациент (0 -- нет, 1 -- да)',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Признак `obesity`

**Комментарий**: рассмотрим признак `obesity` -- страдает ли пациент от ожирения.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['obesity'],
                   title = 'Страдает ли пациент от ожирения?',
                   x_label = 'Ожирение (0 -- нет, 1 -- да)',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'obesity', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от ожирения',
                       x_label='Ожирение(0 -- нет, 1 -- да)',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Признак `alcohol_consumption`

**Комментарий**: рассмотрим признак `alcohol_consumption` -- употребление алкоголя пациентом.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['alcohol_consumption'],
                   title = 'Употребление алкоголя пациентом',
                   x_label = 'Употребляет ли алкоголь пациент (0 -- нет, 1 -- да)',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'alcohol_consumption', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от употребления алкоголя',
                       x_label='Употребляет ли алкоголь пациент (0 -- нет, 1 -- да)',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Признак `diet`

**Комментарий**: рассмотрим признак `diet` -- какая у пациента диета/питание.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['diet'],
                   title = 'Диета/тип питания у пациента',
                   x_label = 'Тип диеты/питания',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'diet', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от диеты/питания',
                       x_label='Тип диеты/питания',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Признак `medication_use`

**Комментарий**: рассмотрим признак `medication_use` -- принимает ли пациент лекарственные препараты.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['medication_use'],
                   title = 'Принимает ли пациент лекарственные препараты',
                   x_label = 'Принимает ли пациент лекарства (0 -- нет, 1 -- да)',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'medication_use', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от приема лекарств',
                       x_label='Принимает ли пациент лекарства (0 -- нет, 1 -- да)',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Признак `stress_level`

**Комментарий**: рассмотрим признак `stress_level` -- уровень стресса пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['stress_level'],
                   title = 'Уровень стресса пациента',
                   x_label = 'Уровень стресса',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart, 
                       'stress_level', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от уровня стресса',
                       x_label='Уровень стресса',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Признак `gender`

**Комментарий**: рассмотрим признак `gender` -- пол пациента.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['gender'],
                   title = 'Пол пациента',
                   x_label = 'Пол',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

**Комментарий**: 

In [ ]:
# посмотрим распределение в зависимости от целевого
plot_feature_vs_target(heart,
                       'gender', 
                       'heart_attack_risk_binary',
                       title='Количество пациентов в зоне риска в зависимости от пола пациента',
                       x_label='Пол',
                       y_label='Количество пациентов'
                       )

**Промежуточный вывод**:

#### Целевой признак `heart_attack_risk_binary`

**Комментарий**: рассмотрим **целевой признак** `heart_attack_risk_binary` -- риск сердечного приступа.

In [ ]:
# выведем график распределения
plot_data_analysis(heart['heart_attack_risk_binary'],
                   title = 'Количество пациентов с риском сердечного присута',
                   x_label = 'Риск (0 -- нет, 1 -- да)',
                   y_label = 'Количество пациентов',
                   plot_bar = True,
                   plot_box = False,
                   discrete = True
                   )

In [ ]:
heart['heart_attack_risk_binary'].value_counts()

**Комментарий**: наблюдается дисбаланс в целевом признаке. Важно это учитывать при формировании выборки (стратификация), и при настройке и обучении моделей (например, балансировочные веса для бустинга и деревьев).

**Промежуточный вывод**:

### Промежуточный вывод

## <u>Корреляционный анализ</u>

**Комментарий**: выполним корреляционный анализ признаков.

In [ ]:
# составим список числовых столбцов
corr_columns = heart.select_dtypes(include=np.number).columns.tolist()
# построим диаграммы рассеяния
#sns.pairplot(heart[corr_columns], hue='heart_attack_risk_binary');

In [ ]:
# сформируем список столбцов с непрерывными признаками
interval_columns = ['age', 
                    'cholesterol', 
                    'heart_rate', 
                    'exercise_hours_per_week',
                    'sedentary_hours_per_day',
                    'income',
                    'bmi',
                    'triglycerides',
                    'blood_sugar',
                    'systolic_blood_pressure',
                    'diastolic_blood_pressure'
                    ]

🤖 -- это МЕГАТРОН, он тут приглядывает за всем.

In [ ]:
# рассчитаем корреляцию и построим матрицу коэффициентов корреляции и тепловую карту
corr_matrix = plot_corr_heatmap(heart,
                                title = 'Коэффициенты корреляции в данных heart_train',
                                columns = interval_columns)
display(corr_matrix)

In [ ]:
# проведем беглый анализ корреляционной матрицы
corr_res = corr_analysis(corr_matrix, target_feature='heart_attack_risk_binary', min_abs_corr=1e-8)

In [ ]:
# выведем результаты анализа корреляционной матрицы
display(corr_res)

**Комментарий**: слабые еле уловимые связи, нужно смотреть `SHAP` и `feature_importance`. Также наблюдается аномальная корреляция между признаками `smoking` и `gender`. Исследуем этот момент подробнее.

In [ ]:
plt.figure(figsize=(13, 5))

# гистограмма для курящих
plt.hist(heart[heart['smoking'] == 1]['age'], 
         alpha=0.8, label='Курит', bins=20, color='salmon', edgecolor='black')

# гистограмма для некурящих  
plt.hist(heart[heart['smoking'] == 0]['age'],
         alpha=0.8, label='Не курит', bins=20, color='lightblue', edgecolor='black')

plt.xlabel('Нормализованный возраст')
plt.ylabel('Количество пациентов')
plt.title('Распределение курения по возрасту')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# получаем распределение по полу
smoking_by_gender = pd.crosstab(heart['gender'], heart['smoking'], normalize='index') * 100
print("Распределение курения по полу (%):")
print(smoking_by_gender)

# считаем абсолютные значения
smoking_counts = pd.crosstab(heart['gender'], heart['smoking'])
print("\nАбсолютные значения:")
print(smoking_counts)

# строим график
plt.figure(figsize=(13, 5))
sns.heatmap(smoking_counts, annot=True, fmt='d', cmap='YlOrRd')
plt.title('Распределение курения по полу')
plt.show()

**Комментарий**: признак `smoking` следует исключить в связи с обнаружением существенного дисбаланса: **100**% мужчин в выборке являются курильщиками (по `gender`) и также все, у кого возраст $>0.3$ являются курильщиками.

In [ ]:
# рассчитаем корреляцию и построим матрицу коэффициентов корреляции и тепловую карту
corr_matrix = plot_corr_heatmap(heart,
                                title = 'Коэффициенты корреляции в данных heart_train',
                                columns = interval_columns)
display(corr_matrix)

## <u>Группировка признаков</u>

In [ ]:
heart['anthropometric_score'] = (
    (heart['bmi'] > 0.5).astype(int) +
    heart['obesity'] +
    (heart['age'] > 0.4).astype(int)
)

In [ ]:
heart['lifestyle_score'] = (
    heart['smoking'] +                                    # Курение = +1
    (heart['stress_level'] > 7).astype(int) +             # Высокий стресс (8-10) = +1
    heart['alcohol_consumption'] +                        # Алкоголь = +1
    (heart['exercise_hours_per_week'] < 0.5).astype(int) + # Мало спорта = +1
    (heart['sedentary_hours_per_day'] > 0.5).astype(int) + # Сидячий = +1
    (heart['physical_activity_days_per_week'] < 3).astype(int) + # < 3 дней активности = +1
    (heart['sleep_hours_per_day'] < 0.5).astype(int) +    # Мало сна = +1
    (heart['diet'] == 0).astype(int) +                    # Плохая диета = +1
    (heart['income'] < 0.5).astype(int)
)

In [ ]:
heart['test_score'] = (
    (heart['cholesterol'] > 0.5).astype(int) +      # Высокий холестерин = +1
    (heart['triglycerides'] > 0.5).astype(int) +    # Высокие триглицериды = +1
    (heart['blood_sugar'] > 0.5).astype(int)        # Высокий сахар = +1
)

In [ ]:
heart['medical_score'] = (
    heart['medication_use'] +
    heart['family_history'] +
    heart['diabetes']
)

In [ ]:
heart.drop(columns=['blood_sugar', 
                    'cholesterol', 
                    'triglycerides',
                    'smoking',
                    'stress_level',
                    'alcohol_consumption',
                    'exercise_hours_per_week',
                    'sedentary_hours_per_day',
                    'physical_activity_days_per_week',
                    'sleep_hours_per_day',
                    'diet',
                    'income',
                    'bmi',
                    'obesity',
                    'age',
                    'diabetes',
                    'medication_use',
                    'family_history'], inplace=True)

In [ ]:
# рассчитаем корреляцию и построим матрицу коэффициентов корреляции и тепловую карту
corr_matrix = plot_corr_heatmap(heart,
                                title = 'Коэффициенты корреляции в данных heart_train',
                                columns = interval_columns)
display(corr_matrix)

## <u>Обучение модели</u>

**Комментарий**:

### Подготовка данных

**Комментарий**: т.к. у нас имеется изначально тренировочная выборка, то предлагается разбить ее на подвыборки -- обучающую и валидационную.

In [ ]:
X = heart.drop('heart_attack_risk_binary', axis=1)
y = heart['heart_attack_risk_binary']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE, 
    stratify=y  # сохраняем пропорции целевого признака
)

print(f"Размер обучающей выборки: {len(X_train)}")
print(f"Размер валидационной выборки: {len(X_test)}\n")
print(f"Пропроция целевого признака в обучающей выборке:\n{y_train.value_counts(normalize=True)}\n")
print(f"Пропроция целевого признака в валидационной выборке:\n{y_test.value_counts(normalize=True)}")

In [ ]:
# формируем списки признаков
num_columns = heart.drop(columns=['gender', 'heart_attack_risk_binary']).columns.tolist()
cat_columns = ['gender']

**Комментарий**:

In [ ]:
# ohe enconder
categorical_transformer_ohe = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'))
])

# ordinal encoder
categorical_transformer_ordinal = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# количественные признаки
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# два препроцессора
data_preprocessor_ohe = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_columns),
        ('cat', categorical_transformer_ohe, cat_columns)
    ])

data_preprocessor_ordinal = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_columns),
        ('cat', categorical_transformer_ordinal, cat_columns)
    ])

**Промежуточный вывод**:

### Выбор метрики

В качестве метрик будем рассматривать следующие:
- ```f1-score``` - баланс между ```precision``` и ```recall``` в условиях дисбаланса классов;
- ```recall``` (полнота) -- минимизация ложноотрицательных случаев, не пропускаем реальные случаи проблем с сердцем. **Т.к. несвоевременное предсказание проблем с сердцем может оказаться фатальным, то нужно максимизировать ```recall```**.

В качестве общих дополнительных рассмотреть можно учитывать еще:
- ```ROC-AUC``` - общее качество разделения классов;
- ```precision``` (точность) - минимизация ложноположительных случаев на втором плане, лучше перебдеть конечно;
- ```average precision``` (**```ROC-PR```**) - лучше при дисбалансе целевого признака (у нас наблюдается дисбаланс $64$%/$36$%), в этом случае больше внимания уделяется риску проблем с сердцем. Для медицинской задачи, коей является цель настоящего проекта, это будет более строгая метрика.


### Обучение модели

**Комментарий**:

In [ ]:
# составим список моделей и наборов гиперпараметров
scalers = {
    'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
}
models = [
    {
        'model': DummyClassifier(),
        'params': {
            'model__strategy': ['stratified', 'most_frequent', 'prior', 'uniform'],
            **scalers
        },
        'name': 'Dummy'
    },
    {
        'model': LogisticRegression(random_state=RANDOM_STATE),
        'params': {
            'model__C': [0.1, 1.0, 10.0],
            'model__penalty': ['l1', 'l2', 'elasticnet'],
            'model__solver': ['liblinear', 'saga'],
            'model__max_iter': [1000],
            'model__class_weight': ['balanced', None],
            **scalers
        },
        'name': 'LogisticRegression'
    },
    {
        'model': DecisionTreeClassifier(random_state=RANDOM_STATE),
        'params': {
            'model__max_depth': [3, 5, 7, 10],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__criterion': ['gini', 'entropy'],
            'model__class_weight': ['balanced', None],
            **scalers
        },
        'name': 'DecisionTreeClassifier'
    },
    {
        'model': GradientBoostingClassifier(random_state=RANDOM_STATE),
        'params': {
            'model__n_estimators': [50, 100, 200, 500, 1000],
            'model__learning_rate': [0.01, 0.1, 0.2],
            'model__max_depth': [3, 4, 5],
            'model__subsample': [0.8, 1.0],
            **scalers
        },
        'name': 'GradientBoostingClassifier'
    },
    {
        'model': KNeighborsClassifier(),
        'params': {
            'model__n_neighbors': [3, 5, 7],
            'model__weights': ['uniform', 'distance'],
            'model__metric': ['euclidean', 'manhattan', 'minkowski'],
            **scalers
        },
        'name': 'KNeighborsClassifier'
    },
    {
        'model': RandomForestClassifier(random_state=RANDOM_STATE),
        'params': {
            'model__n_estimators': [100, 200, 300, 500, 1000],
            'model__max_depth': [5, 10, 15, None],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__class_weight': ['balanced', None],
            **scalers
        },
        'name': 'RandomForestClassifier'
    },
    {
        'model': XGBClassifier(random_state=RANDOM_STATE, n_jobs=-1),
        'params': {
            'model__n_estimators': [100, 200, 300, 500, 1000],
            'model__max_depth': [3, 6, 9],
            'model__learning_rate': [0.01, 0.1, 0.2],
            'model__subsample': [0.8, 0.9, 1.0],
            'model__colsample_bytree': [0.8, 0.9, 1.0],
            'model__scale_pos_weight': [1, len(y_train[y_train==0])/len(y_train[y_train==1])],
            'model__verbose': [0],
            **scalers
        },
        'name': 'XGBClassifier'
    },
    {
        'model': LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1),
        'params': {
            'model__n_estimators': [100, 200, 300, 500, 1000],
            'model__max_depth': [3, 5, 7, -1],
            'model__learning_rate': [0.01, 0.1, 0.2],
            'model__num_leaves': [31, 63, 127],
            'model__min_child_samples': [1, 5, 10],
            'model__min_split_gain': [0.0, 0.1],
            'model__class_weight': ['balanced', None],
            'model__verbose': [-1],
            **scalers
        },
        'name': 'LGBMClassifier'
    },
    {
        'model': SVC(random_state=RANDOM_STATE, probability=True),
        'params': {
            'model__C': [0.1, 1.0, 10.0],
            'model__kernel': ['linear', 'rbf', 'poly'],
            'model__gamma': ['scale', 'auto'],
            'model__class_weight': ['balanced', None],
            **scalers
        },
        'name': 'SVC'
    },
    {
    'model': CatBoostClassifier(random_state=RANDOM_STATE, verbose=False),
    'params': {
        'model__iterations': [100, 200, 500, 1000],
        'model__depth': [4, 6, 8, 10, 12],
        'model__learning_rate': [0.01, 0.03, 0.05, 0.1],
        'model__l2_leaf_reg': [1, 3, 5],
        'model__border_count': [32, 64, 128],
        **scalers
    },
    'name': 'CatBoostClassifier'
}
]

In [ ]:
# в цикле выполним подбор параметров, замеряем время обучения, предсказания и подборов параметров
results = []
for mdl in models:
    results.append(evaluate_model(mdl['model'], 
                                  mdl['params'], 
                                  mdl['name'],
                                  X_train,
                                  y_train,
                                  data_preprocessor_ohe,
                                  data_preprocessor_ordinal,
                                  refit_metric='roc_auc',
                                  random_state = RANDOM_STATE,
                                  cv=5,
                                  n_iter=10))

In [ ]:
# сформируем DataFrame из итоговых результатов и выведем таблицу
results_df = pd.DataFrame(results).copy()
if not results_df.empty:
    # выведем таблицу с результатами по всем моделям
    print("Итоговые результаты:")
    display(results_df[['model_name', 
                    'accuracy_cv', 
                    'precision_cv',
                    'recall_cv',
                    'roc_auc_cv',
                    'average_precision_cv',
                    'f1_cv',
                    'params_time',
                    'train_time',
                    'predict_time']].sort_values(['f1_cv'],
                                                 ascending=False).reset_index(drop=True))

    # лучшая модель по recall
    best_model_metrics = results_df.sort_values(['f1_cv'],
                                                ascending=False).iloc[0]

    # выводим результат
    print(f"Лучшая модель: {best_model_metrics['model_name']}") 
    print(f"Average Precision на кросс-валидации = {best_model_metrics['average_precision_cv']:.4f}")
    print(f"Recall на кросс-валидации = {best_model_metrics['recall_cv']:.4f}")
    print(f"ROC-AUC на кросс-валидации = {best_model_metrics['roc_auc_cv']:.4f}")
    print(f"F1-score на кросс-валидации = {best_model_metrics['f1_cv']:.4f}")
    print(f"Accuracy на кросс-валидации = {best_model_metrics['accuracy_cv']:.4f}")
    print(f"Precision на кросс-валидации = {best_model_metrics['precision_cv']:.4f}")
    print(f"train_time = {best_model_metrics['train_time']:.3f} с.")
    print(f"predict_time = {best_model_metrics['predict_time']:.3f} c.")

    best_final_model = best_model_metrics['best_model']
else:
    print("Ошибка в получении модели!")

**Промежуточный вывод**:

## <u>Тестирование модели</u>

**Комментарий**: выполним тестирование лучшей найденной модели на валидационной выборке. 

In [ ]:
# выведем модель
print(best_final_model)

In [ ]:
y_pred = best_final_model.predict(X_test)
y_pred_proba = best_final_model.predict_proba(X_test)[:, 1]  # вероятности для положительного класса

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
average_precision = average_precision_score(y_test, y_pred_proba)

print(f"Метрики на тестовой выборке для финальной модели {best_model_metrics['model_name']}:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Average Precision: {average_precision:.4f}")

# выведем матрицу ошибок
print("\nМатрица ошибок:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

plot_confusion_matrix(cm)

In [ ]:
if hasattr(best_final_model.named_steps['model'], 'predict_proba'):
    y_pred_proba = best_final_model.predict_proba(X_test)[:, 1]
else:
    # для SVC используем decision_function
    y_pred_proba = best_final_model.decision_function(X_test)
    # нормализуем к [0, 1]
    y_pred_proba = (y_pred_proba - y_pred_proba.min()) / (y_pred_proba.max() - y_pred_proba.min())

# строим PR curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

# строим график
plt.figure(figsize=(13, 6))
plt.plot(recall, precision)
plt.xlabel('recall')
plt.ylabel('precision')
plt.title('Кривая precision-recall')
plt.grid(True)
plt.show()

# смотрим варианты порогов
results_r = []
for target_recall in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 1.0]:
    # находим ближайший порог
    idx = np.argmin(np.abs(recall[:-1] - target_recall))
    threshold = thresholds[idx]
    
    # применяем порог
    y_pred_custom = (y_pred_proba > threshold).astype(int)
    
    # cчитаем метрики
    prec = precision_score(y_test, y_pred_custom)
    rec = recall_score(y_test, y_pred_custom)
    f1 = f1_score(y_test, y_pred_custom)
    
    results_r.append({
        'target_recall': target_recall,
        'threshold': threshold,
        'precision': prec,
        'recall': rec,
        'f1': f1
    })
    
    print(f"Recall {target_recall}: threshold={threshold:.3f}, precision={prec:.3f}, f1={f1:.3f}")


## Промежуточный вывод

## <u>Анализ фажности признаков</u>

**Комментарий**: выполним теперь анализ важности признаков

In [ ]:
# получаем имена 
feature_names = best_final_model.named_steps['preprocessor'].get_feature_names_out()
    
# подготавливаем данные
X_sample = best_final_model.named_steps['preprocessor'].transform(X_test)
  
model = best_final_model.named_steps['model']
model_name = best_model_metrics['model_name']

In [ ]:
# feature importance (стандартная)
if hasattr(model, 'feature_importances_'):
    importance = model.feature_importances_
        
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False)
        
    plt.figure(figsize=(13, 6))
    sns.barplot(x='importance', y='feature', data=importance_df.head(20))
    plt.title(f'Важность признаков ({best_model_metrics["model_name"]})')
    plt.show()
        
    print("\nТоп-20 важных признаков:")
    print(importance_df.head(20))
else:
    print(f"Модель {model_name} не имеет атрибута feature_importances_")

In [ ]:
print(f"\nSHAP анализ для {best_model_metrics['model_name']}:")
try:
    if any(tree_model in model_name for tree_model in 
           ['RandomForest', 'DecisionTree', 'XGB', 'LGBM', 'CatBoost', 'GradientBoosting']):
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_sample)
        
    elif 'SVC' in model_name:
        background = shap.sample(X_sample, 100)
        explainer = shap.KernelExplainer(model.predict, background)
        shap_values = explainer.shap_values(X_sample)
        
    elif 'LogisticRegression' in model_name:
        explainer = shap.LinearExplainer(model, X_sample)
        shap_values = explainer.shap_values(X_sample)
        
    else:
        explainer = shap.Explainer(model, X_sample)
        shap_values = explainer(X_sample)
    
    print(f"Используется {type(explainer).__name__} для {model_name}")
    
    # summary plot
    shap.summary_plot(shap_values, X_sample, feature_names=feature_names, show=False)
    plt.gcf().set_size_inches(13, 6)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Ошибка при SHAP анализе: {str(e)}")

## <u>Сохранение итоговой модели</u>

## <u>Итоговый вывод</u>